## Protein Generation and Filtering with BioNeMo

This example notebook shows how to generate protein sequences using ProtGPT2 inference from BioNemo service API. For more details, please visit the NVIDIA BioNeMo Service homepage at https://www.nvidia.com/en-us/gpu-cloud/bionemo/

This notebook will walk through a protein generation and filtering workflow in the following sections:

 - **BioNeMo Service Configuration**
   - Install dependencies and define the BioNeMo service endpoint and API key required for access
 - **Generating Protein Sequences**
   - Use BioNeMo Service to generate protein sequences with the ProtGPT2 model
 - **Predict the properties of the generated sequences**
   - Use [PGP](https://github.com/hefeda/PGP) to understand the properties of the generated sequences

### BioNeMo Service Configurations
To get started, please configure and provide your NGC access token by visiting https://ngc.nvidia.com/setup/api-key|

In [ ]:
API_KEY="Your Key Here"
API_HOST="https://api.stg.bionemo.ngc.nvidia.com/v1"

Let's start by installing and importing library dependencies. We'll also use _requests_ to interact with the BioNeMo service, and _BioPython_ to parse FASTA sequences into SeqRecord objects.

In [ ]:
!pip install torch biopython

In [ ]:
import requests
import time
import json
import pathlib

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

Next let's verify connectivity to the BioNeMo service. You should get a "200 OK" HTTP response in the following cell, indicating a successful connection. Note the _Authorization_ field in the header response which contains your NGC access token credentials. We'll re-use this header for all future interactions with the service.

<div class="alert alert-block alert-info">
    <b>Tip:</b>  If you do not receive a "200 OK" HTTP response when testing connectivity to BioNeMo service in the cell below, verify
    
 - the API_HOST defined above is the correct address of the BioNeMo service, and
 - the API_KEY is authorized to access BioNeMo service at the API_HOST

Refer to this <a href="https://developer.mozilla.org/en-US/docs/Web/HTTP/Status">list of HTTP responses status codes</a> to help determine the cause of a non-200 response.
</div>

In [ ]:
#Checking to see if the access is configured
response = requests.get(
    f"{API_HOST}/models",
    headers={"Authorization": f"Bearer {API_KEY}"})
print("Query BioNeMo Service:", response)

#Add key to headers for remainder of notebook
headers = {
    'Authorization': f'Bearer {API_KEY}'
}

Some of the BioNeMo services as are _sychronous_, meaning the service API call will block until a result is returned. Two examples of synchronous services are MegaMolBART molecule embedding, and looking up PDBs using the BioNeMo Uniprot service. Other services are _asynchronous_, such as the protein folding and docking services.

You can find more information about the BioNeMo API here: https://bionemo.ngc.nvidia.com/openapi

Functions calling _asynchronous_ services are non-blocking, and immediately return a handle called _correlation_id_ that can be used to query the results. This allows us to batch multiple requests together and query them for completion at a later time.

In the following cell we introduce a helper function _query_async_result_ that will block on a _correlation_id_ task until the computation is completely and a result is returned.  For these non-blocking asynchronous calls, it is important to block on the correlation id to ensure we've received the response data before continuing.

In [ ]:
def query_async_result(request, print_result=False):
    if isinstance(request, str):
        #Request is a correlation id string
        correlation_id=request
    elif isinstance(request, requests.models.Response):
        submission_response = json.loads(request.content)
        correlation_id=submission_response['correlation_id']
    
    i = 0
    while True:
        response = requests.get(
            f"{API_HOST}/task/{correlation_id}",
            headers=headers,
        )      
        
        status_result = json.loads(response.content)
        if status_result['control_info']['status'] == 'DONE':
            if(print_result): print(status_result['response'])
            return(status_result['response'])
        if status_result['control_info']['status'] == 'ERROR':
            print("ERROR, Cancelling Result Retrieval")
            return            
        else:
            print(f"{status_result['control_info']['status']}{''.join(['.']*i)}" , end="\r")
            time.sleep(10)  # waiting for the prediction from BioNeMo Server
        i = i+1

## Unconditionally generating protein sequences
BioNeMo exposes [ProtGPT2](https://www.nature.com/articles/s41467-022-32007-7) as a service for unconditional protein generation. The service can be queried to generate sequences following natural sequence distributions as present in UniProt. These sequences can then be used as the starting point for a [drug discovery campaign](https://www.sciencedirect.com/science/article/pii/S2001037022005086#s0020), which we'll look into in the next chapters of this notebook.

It's important to choose your parameters for the protein generation process carefully.

-   max_length: maximum number of generated tokens.
        
Note that common tokens in ProtGPT3 are k-mers of length 3, so 1 token = 3 amino acids. In other words, a max_length=400 could translate to an effective protein sequence of up to 1200 amino acids.

- top_k: Sets the number of highest probability vocabulary tokens to keep for top-k-filtering
- repetition_penalty: Sets the penalty for [repeating tokens](https://arxiv.org/pdf/1909.05858.pdf), where a value of 1 correspond to no penalty.
- num_return_sequences: The effective number of whole protein sequences to return
- percent_to_keep: Sets the percent of whole protein sequences to keep for each protein generation iteration cycle, based on their cumulative perplexity. 

As an illustrative example, suppose that 50 sequences of tokens are generated per itration. From these generated sequences of tokens (which could contain protein sequence fragments or other invalid # sequences), whole protein sequences are reconstructed. Let's suppose that there are 43 valid, whole protein sequences from the generative iteration. Of these 43 whole protein sequences, only the top `percent_to_keep`, according to perplexity, are kept. For instance, with percent_to_keep=10, only ~4 of the 43 sequences (i.e., ~10% of 43) will be kept. After these 4 sequences are added to the return object, the generation process will resume until `num_return_sequences` is reached. Therefore - the lower the value of `percent_to_keep`, the longer the overall generation process will take.

Now let's define a sane set of parameters for our example problem.

In [ ]:
# Here we define the generation parameters.
parameters = {
    "max_length": 400,
    "top_k": 950,
    "repetition_penalty": 1.2,
    "num_return_sequences": 100,
    "percent_to_keep": 10
}

Now we can submit our service request to BioNeMo. Even though this service is asynchronous, we'll block on waiting for our request to be processed.

In [ ]:
# Submit request ticket
submission_request = requests.post(
    f"{API_HOST}/protein-sequence/protgpt2/generate",
    headers=headers,
    json=parameters
)

# Wait for request to be processed, and convert the result to a dictionary
result = json.loads(query_async_result(submission_request))

Next let's convert the generated sequences into SeqRecord items, so that we can store them in FASTA file locally for later use.

In [ ]:
# Process the request response into biopython seqrecords
seqrecords = list()

for i, (perplexity, sequence) in enumerate(
    zip(result['perplexities'], result['generated_sequences'])
):
    seqrecords.append(SeqRecord(id=f"S{i+1}", seq=Seq(sequence), description=f", L={len(sequence)}, ppl={perplexity}"))
    
# Write the sequences to a fasta file
sequences_filename = "sequences.fasta"
with open(sequences_filename, "w") as output_handle:
    SeqIO.write(seqrecords, output_handle, "fasta")   

Let's take a look at the saved FASTA file for a quick sanity check.

In [ ]:
#Print header lines from protein FASTA file
!head sequences.fasta

## Predicting properties from generated protein sequences using PGP
[PGP](https://github.com/hefeda/PGP) is an open-source tool that can be used to rapidly analyze protein sequences to predict properties such as secondary structure, gene ontology, subcellular localization, and [more](https://www.sciencedirect.com/science/article/pii/S2001037022005086#s0020).

We will use [PGP](https://github.com/hefeda/PGP) here to analyze the sequences generated with ProtGPT2 above to get a better understanding of their properties and the occurrence of desired features.

To begin, we need to clone the PGP repository locally and install its dependencies and run the predictors.

In [ ]:
# Clone the PGP repository locally
!git clone https://github.com/hefeda/PGP.git

PGP relies on _torch_. If the library isn't already installed on your system, expect installation to take around tens minutes or so.

In [ ]:
# Install the minimal requirements
!pip install -r PGP/requirements_minimal.txt

In [ ]:
# Predict features using PGP
!python PGP/prott5_batch_predictor.py --input sequences.fasta --output predictions --fmt ss,cons,dis,mem,bind,go,subcell,tucker

## Reading predictions and querying sequences based on desired properties
Lastly, we will load the predicted properties and use them to query the sequences for desired features.

In [ ]:
from PGP.notebooks.utils import load_data

In [ ]:
predicted_features = load_data(pathlib.Path('predictions'))

## Short sequences without transmembrane strands, binding to small molecules

In the example above, we limited our ProtGPT2 request to generate only 100 sequences to minimize runtime.  In a production workflow, we would typically work with more complex queries on a much larger generated dataset of sequences.  To increase the likelihood of finding a complex, pharmacologically relevant sequence in our smaller sample set, we will query for sequences that:
- Are longer than 100 residues
- Do not include transmembrane strand content
- Bind to small molecules

Lastly we return the sequences ordered by longest sequences with most transmembrane and small molecule binding content. We will print the first ten sequences here. Take a look at the query body for some of the example features that can be inspected, and feel free to manipulate them to change your filtered sequence set.

In [ ]:
filtering_order = ['length', 'transmembrane_strand_percent', 'small_percent']

for sequence in predicted_features.query(
    '''
    length > 100 and\
    transmembrane_strand_count == 0  and\
    small_count > 0 
    '''
)[:9].sort_values(filtering_order, ascending=False).to_records():
    print(f"Header: {sequence.header}")
    print(f"Sequence length: {sequence.length}")
    print(f"Transmembrane strand content: {sequence.transmembrane_strand_percent*100:0.2f}%")
    print(f"Small-molecule binding content: {sequence.small_percent*100:0.2f}%")
    print("---------------")

## Conclusion
In this notebook, we've walked through an example workflow generating protein sequences and analyzing their properties for desired features, covering:

 - How to configure the BioNeMo Service API
 - Generating protein sequences with the ProtGPT2 model
 - Understand the properties of the generated sequences using PGP
   
While this notebook demonstrates some of the rich capabilities of the BioNeMo service, this would just be the beginning of a production end-to-end drug discovery pipeline. As a next step, consider using the BioNeMo folding services to predict the structures of the sequences you generated!